In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!pip install transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00


In [2]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [1]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay


In [2]:
model_checkpoint = 'Helsinki-NLP/opus-mt-en-hi'

In [3]:
dataset = load_dataset('cfilt/iitb-english-hindi')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

Preproccessing the data


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
tokenizer('hello, damn what are you doing?')

{'input_ids': [39915, 2, 22260, 117, 54, 27, 952, 22, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
max_input_length = 128
max_target_length = 128

source_lan = 'en'
target_lan = 'hi'

def preprocess_function(examples):
  inputs = [ex[source_lan] for ex in examples['translation']]
  targets = [ex[target_lan] for ex in examples['translation']]
  model_inputs = tokenizer(inputs,max_length = max_input_length, truncation=True)

  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets,max_length = max_target_length, truncation=True)

  model_inputs['labels'] = labels['input_ids']

  return model_inputs



In [8]:
dataset['train'][:2]

{'translation': [{'en': 'Give your application an accessibility workout',
   'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
  {'en': 'Accerciser Accessibility Explorer',
   'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}]}

In [9]:
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [[3872, 85, 2501, 132, 15441, 36398, 0], [32643, 28541, 36253, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[63, 2025, 18, 16155, 346, 20311, 24, 2279, 679, 0], [26618, 16155, 346, 33383, 0]]}

In [10]:
tokenized_data = dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [11]:
tokenized_data.save_to_disk("tokenized_data")

Saving the dataset (0/2 shards):   0%|          | 0/1659083 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/520 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2507 [00:00<?, ? examples/s]

In [12]:
from datasets import load_from_disk

tokenized_data = load_from_disk("tokenized_data")

In [13]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [14]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epoch = 1

In [15]:
datacollator = DataCollatorForSeq2Seq(tokenizer,model,return_tensors='tf')

In [16]:
generation_data_collector = DataCollatorForSeq2Seq(tokenizer,model,return_tensors='tf',pad_to_multiple_of=128)

In [ ]:
%pip install datasets

In [17]:
tokenized_data['train'][1]

{'translation': {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 'input_ids': [32643, 28541, 36253, 0],
 'attention_mask': [1, 1, 1, 1],
 'labels': [26618, 16155, 346, 33383, 0]}

In [18]:
from IPython import get_ipython
get_ipython().run_line_magic('reload_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

import datasets

In [19]:


train_dataset = model.prepare_tf_dataset(
    tokenized_data['test'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=datacollator
)

In [20]:
validation_dataset = model.prepare_tf_dataset(
    tokenized_data['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=datacollator
)

In [21]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_data['validation'],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collector
)

In [22]:
optimizer=AdamWeightDecay(learning_rate=learning_rate,weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [23]:
model.fit(train_dataset,validation_data=validation_dataset,epochs=num_train_epoch)

156/156 [==============================] - 2184s 14s/step - loss: 3.7629 - val_loss: 3.9471


In [24]:
model.save_pretrained("my_trained_model")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:393: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


Model Testing

In [25]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("my_trained_model")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at my_trained_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [30]:
tokenizer.save_pretrained("tokenizer_directory")

('tokenizer_directory/tokenizer_config.json',
 'tokenizer_directory/special_tokens_map.json',
 'tokenizer_directory/vocab.json',
 'tokenizer_directory/source.spm',
 'tokenizer_directory/target.spm',
 'tokenizer_directory/added_tokens.json')

In [28]:
input_text = 'bro! what is your name'
tokenized_data  = tokenizer([input_text],return_tensors='np')
out = model.generate(**tokenized_data,max_length=128)
print(out)

tf.Tensor([[61949   443    61  1157   179    65     5    22     0]], shape=(1, 9), dtype=int32)


In [29]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0],skip_special_tokens=True))

भाई! आपका नाम क्या है?


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [31]:
!zip -r my_trained_model.zip my_trained_model

  adding: my_trained_model/ (stored 0%)
  adding: my_trained_model/tf_model.h5 (deflated 7%)
  adding: my_trained_model/config.json (deflated 61%)
  adding: my_trained_model/generation_config.json (deflated 43%)


In [32]:
from google.colab import files
files.download('my_trained_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
!zip -r tokenizer_directory.zip tokenizer_directory

  adding: tokenizer_directory/ (stored 0%)
  adding: tokenizer_directory/tokenizer_config.json (deflated 68%)
  adding: tokenizer_directory/special_tokens_map.json (deflated 35%)
  adding: tokenizer_directory/source.spm (deflated 51%)
  adding: tokenizer_directory/vocab.json (deflated 76%)
  adding: tokenizer_directory/target.spm (deflated 60%)


In [34]:
from google.colab import files
files.download('tokenizer_directory.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>